In [3]:
country = "ecuador"

import io,  unicodedata, os, re, urllib3, requests, datetime, re
import pandas as pd ; import numpy as np
from datetime import date
from datetime import datetime
from bs4 import BeautifulSoup
from io import BytesIO

d = os.getcwd()+"/../"
PATH_RAW = f"{d}/raw"
PATH_FORMAT = f"{d}/data"
folder_spec = PATH_RAW

from urllib.request import urlopen

hdr = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537'}
urllib3.disable_warnings()

import tika
from tika import parser    
import tabula
import zipfile
import xlrd

## BCE website

- GDP

PIB: Acceder al website del BCE/Estadisticas/Sector Real https://www.bce.fin.ec/informacioneconomica/sector-real

Cuentas Nacionales > Base Movil 2018 > Cuentas Nacionales Trimestrales https://contenido.bce.fin.ec/documentos/informacioneconomica/cuentasnacionales/ix_cuentasnacionalestrimestrales.html

- Exportaciones mineras

- Exportaciones de hidrocarburos


Extraigo datos de Niveles de Volumen Encadenados, 2018=100 Miles de US Dolares adjustados de estacionalidad

In [ ]:
df0 = pd.read_csv(f'{PATH_RAW}/bce_gdp.csv', parse_dates=True, index_col='date')

url = "https://contenido.bce.fin.ec/documentos/informacioneconomica/cuentasnacionales/ix_cuentasnacionalestrimestrales.html"

r = requests.get(url, headers=hdr, verify=False)
r = r.content

soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

excel_links = [ link.get('href') for link in links if "xls" in link.get('href') and "tou" in link.get('href') ]
excel_links = [ link for link in excel_links if "prel" not in link ]
excel_links

latest = ""
for link in excel_links :
    if latest == "" : latest = link
    else :
        if int(link.split("_")[-2]) > int(latest.split("_")[-2]) : latest = link

from io import BytesIO

df = pd.read_excel( BytesIO(requests.get(latest, headers=hdr, verify=False).content) , sheet_name="Dem_Cad_ajus", skiprows=8, skipfooter=1)
df.dropna(inplace=True)
df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
df.columns = df.columns.str.replace(".","")
df = df[['variables','pib']]
df.variables = df.variables.str.replace(r'[a-z()]','', regex=True)

quarter_mapping = {
    '.IV': '-Q4',
    '.III': '-Q3',
    '.II': '-Q2',
    '.I': '-Q1',    
}

for roman, quarter in quarter_mapping.items():
    df['variables'] = df['variables'].str.replace(roman, quarter)
df.variables = df.variables.str.replace(r'.','').str.strip()

def quarter_to_date(quarter):
    year, q = quarter.split('-')
    month = {'Q1': '01', 'Q2': '04', 'Q3': '07', 'Q4': '10'}[q]
    return pd.to_datetime(f"{year}-{month}-01")

df['date'] = df['variables'].apply(quarter_to_date)
df = df[['date','pib']]
df.set_index('date', inplace=True)
df.rename(columns = { 'pib' : 'gdp_movil' } , inplace=True)

df = pd.merge(df0, df, left_index=True, right_index=True, how='outer', suffixes=('_0', '_1'))
df['gdp_movil'] = df.apply(lambda row: row['gdp_movil_1'] if pd.isna(row['gdp_movil_0']) or row['gdp_movil_0'] != row['gdp_movil_1'] else row['gdp_movil_0'], axis=1)
df = df[['gdp_movil']]

df.to_csv(f'{PATH_RAW}/bce_gdp.csv')
print(f"Updated GDP: {df.index.max()}")

Exportaciones Mineras

In [ ]:
url = "https://contenido.bce.fin.ec/documentos/Estadisticas/Hidrocarburos/ExportacionesAnuales.xlsx"

df = pd.ExcelFile(BytesIO(requests.get(url, headers=hdr, verify=False).content))
df = pd.read_excel(df, sheet_name="Exportaciones Mineras", skiprows=9, header=[0,1,2], skipfooter=10)
df.columns = df.columns.map('{0[0]}_{0[1]}_{0[2]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()

for col in df.columns :
    if "0_level_1" in col :
        df.rename(columns = { col : 'year'} , inplace=True)
    elif "1_level_1" in col :
        df.rename(columns = { col : 'month'} , inplace=True)

df.dropna(subset='month', inplace=True)
df['year'] = df['year'].ffill()

df.month = df.month.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()

df = df[~df['month'].str.contains('-')]

months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12'}
df['month'] = df['month'].astype(str).str.lower().replace(months, regex=True)

df["date"] = pd.to_datetime(df[["year","month"]].assign(DAY=1))

df.set_index('date', inplace=True)
df = df[ [col for col in df.columns if 'kilogram' in col] ]
df.rename(columns=lambda x: x.replace('kilogramos', 'kg'), inplace=True)
df.rename(columns=lambda x: x.replace('_', ' '), inplace=True)
df.rename(columns=lambda x: x.replace('en todas sus formas', ''), inplace=True)
df.rename(columns=lambda x: x.replace('los demas', ''), inplace=True)
df.rename(columns=lambda x: re.sub(r'\s*\(.*?\)\s*', ' ', x), inplace=True)
df.rename(columns=lambda x: re.sub(r'\s{2,}', ' ', x), inplace=True)
df.rename(columns=lambda x: x.replace('*', ''), inplace=True)

df.rename( columns=lambda x: re.sub(r'\b(\w+)\b\s+\b\1\b', r'\1', x ) , inplace=True ) 

df.columns = df.columns.astype("str").str.strip().str.lower()

prefixes = set(col.split(' ')[0] for col in df.columns)
for prefix in prefixes:
    df[f"{prefix} total kg exports"] = df.filter(like=prefix).sum(axis=1)
    
df.to_csv(f"{PATH_RAW}/bce_export.csv")
print(f"Updated Exports: {df.index.max()}")

Exportaciones de hidrocarburos

In [ ]:
url = "https://contenido.bce.fin.ec/documentos/Estadisticas/Hidrocarburos/SerieCifrasPetroleras.xlsx"

df = pd.read_excel( BytesIO(requests.get(url, headers=hdr, verify=False).content) , header=[1,2])
df.columns = df.columns.map('{0[0]}_{0[1]}'.format)

df.rename(columns = {"Unnamed: 0_level_0_Unnamed: 0_level_1" : "var0"}, inplace=True)
df.rename(columns = {"Unnamed: 1_level_0_Unnamed: 1_level_1" : "var1"}, inplace=True)

df.var0 = df.var0.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
df.var1 = df.var1.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()

df['var0'] = np.where(df['2007_ene'].notna(), np.nan, df['var0'])
df['var0'] = df['var0'].ffill()

df = df[~df['var1'].str.contains('tasa de crecimiento', case=False, na=False)]


df["var1"] = df["var1"].str.replace("-", "").str.strip()
v = {'produccion promedio diaria': 'production_bpd_oil',
 'total exportaciones de petroleo' : "export_bdp",
 'gasolina super' : 'production_bpd_gassuper',
 'gasolina extra' : 'production_bpd_gasextra', 
 'extra con ethanol' : 'production_bpd_ecopais',
 'ecoplus' :'production_bpd_ecoplus',
 'diesel' : 'production_bpd_diesel',
 'fuel oil # 4' : 'production_bpd_fueloil4',
 'fuel oil # 6' : 'production_bpd_fueloil6' ,
 'gas licuado de petroleo' : 'production_bpd_lpg',
 'otros' : 'production_bpd_other',
 'consumo promedio diario' : "consumption_bdp",
 'precio' : 'price_oil'
}

df["var1"] = df["var1"].replace(v, regex=True)


df = df[df['var1'].str.contains('production_') | df['var1'].str.contains('price_') | df['var1'].str.contains('consumption_')  | df['var1'].str.contains('export_')]


df = df.loc[ (df.index==2) | (df.index==48) |(df.index>60) & (df.index<69) | (df.index==77) | (df.index == 88) | (df.index == 89) | (df.index == 90) | (df.index == 95) ]

df.var1 = df.var1.str.replace(r'\s*\(.*?\)\s*', '', regex=True)
df.drop('var0', axis=1, inplace=True)

df.rename(columns = {"var1": "variable"}, inplace=True)
df = pd.melt(df, id_vars="variable", var_name="date")

months = {'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12'}
df["date"] = df["date"].astype(str).str.lower().replace(months, regex=True)
df["date"] = df["date"].astype(str).str.lower().replace(r'(\d{4})_\d{4}', r'\1', regex=True)
df = df[~df["date"].str.contains(r'\d{4}_\d{2}.', r'show', regex=True)]
df["date"] = df["date"].astype(str).str.lower().replace(r'(\d{4})_(\d{2})', r'\1-\2-01', regex=True)
df.date = pd.to_datetime(df['date'], format="%Y-%m-%d")
df.set_index('date', inplace=True)

df.dropna(subset='value', inplace=True)

df = pd.pivot(df, columns="variable", values="value")

df.to_csv(f"{PATH_RAW}/bce_oil.csv")
print("Oil updated")
print(df.index.max())


**Expectativas**

- Indice de Expectativas de la Economia
- Indice de Actividad Economica Coyuntural (IDEAC)
- Indice de Confianza del Consumidor


In [ ]:
get = "https://contenido.bce.fin.ec/documentos/PublicacionesNotas/Catalogo/Encuestas/EOE/IEE_Nueva_Metodologia.xlsx"

df = pd.read_excel( BytesIO(requests.get(get, headers=hdr, verify=False).content) , skiprows=7 , parse_dates=True )
df.dropna(inplace = True)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()

df['date'] = pd.to_datetime(df['fecha'])
df.set_index('date', inplace=True)
df.columns = df.columns.str.replace(r'\s*\(.*?\)\s*', '', regex=True)
df.drop('fecha', axis = 1, inplace=True)

df = df.rename(columns=lambda x: f"iee_{x}" if "iee" not in x else x)
df.columns = df.columns.astype("str").str.replace(' ', '_')
df.to_csv(f"{PATH_RAW}/bce_iee.csv")
print("IEE updated")
print(df.index.max())

**Indice de confianza al consumidor**

In [ ]:
url = "https://contenido.bce.fin.ec/documentos/Estadisticas/SectorReal/Previsiones/IndCoyuntura/mensual/ICC_indice.htm"
r = requests.get(url=url, verify=False).content

soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

excel_links = [ link.get('href') for link in links if "icc" in link.get('href').lower() and "pdf" not in link.get('href').lower()]

try :
    df0 = pd.read_csv(f"{PATH_RAW}/bce_icc.csv", parse_dates=True, index_col='date')
except :
    df0 = pd.DataFrame()


latest_url = []
for url in excel_links :
    if len(df0) == 0 : latest_url.append(url)
    elif pd.to_datetime(url.replace('.html','')[-6:], format = "%Y%m") > df0.index.max() :  latest_url.append(url)

for url in latest_url :
    r = requests.get(url=url, verify=False).content
    df = pd.read_html(r)[0][['Indicador Global']]
    df['date'] = pd.to_datetime(url.replace('.html','')[-6:], format = "%Y%m")
    df.set_index('date', inplace=True)
    df.rename( columns = {'Indicador Global' : 'icc'} , inplace=True)
    df['icc'] = df['icc']/100
    df0 = pd.concat([df0,df])

df0.to_csv(f"{PATH_RAW}/bce_icc.csv")
print("ICC updated")
print(df.index.max())

**Reservas Internacionales**

In [ ]:
url = "https://contenido.bce.fin.ec/home1/estadisticas/bolsemanal/IndiceBMS.htm"

r = requests.get(url, headers=hdr, verify=False)
r = r.content

soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

excel_links = [ link.get('href') for link in links if "xls" in link.get('href') ]

def extract_date(url):
    date_match = re.search(r'BMS_(\d{8})', url)
    if date_match:
        date_str = date_match.group(1)
        return datetime.strptime(date_str, '%d%m%Y')
    return None

excel_links = [ (url, extract_date(url) ) for url in excel_links ]
excel_links = [ pair for pair in excel_links if pair[1] is not None ]

get = max(excel_links, key=lambda x: x[1])[0]
df0 = pd.ExcelFile( BytesIO(requests.get(get, headers=hdr, verify=False).content) )

df = pd.read_excel( df0 , sheet_name='IMS1', skiprows=6, skipfooter=13, header=[0,1,2])

df.columns = df.columns.map('{0[0]}_{0[1]}_{0[2]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()

df['year'] = df['periodo_unnamed: 0_level_1_unnamed: 0_level_2'].ffill()
months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }
df['month'] = df['periodo_unnamed: 1_level_1_unnamed: 1_level_2'].astype(str).str.lower().replace(months, regex=True)

df.dropna(subset='year', inplace=True)
df.year = df.year.astype(int)
df['day'] = df['periodo_unnamed: 2_level_1_unnamed: 2_level_2']
df.dropna(subset='day', inplace=True)
df.day = df.day.astype(int)
df['date'] = pd.to_datetime(df[['year', 'month', 'day']], errors='coerce')

df.dropna(subset='date', inplace=True)
df['date'] = pd.to_datetime( df['date'] )
df.columns = df.columns.str.replace(r'\(.*?\)', '', regex=True).str.strip()

df = df[[ col for col in df.columns if 
         "banco central" in col 
         or "panorama financiero" in col 
         or "tasas de interes" in col
         or "date" in col ]]

df.columns = df.columns.str.replace(r'_unnamed: \d+_level_\d+', '', regex=True).str.strip()
df.columns = df.columns.str.replace(r'\nporcentajes_', ' ', regex=True).str.strip()
df.columns = df.columns.str.replace(r'_', ' ', regex=True).str.strip()
df.set_index('date', inplace=True)

# Make all columns into numeric
df = df.applymap(lambda x: pd.to_numeric(x, errors='coerce'))
df.dropna(axis=1, how='all', inplace=True)

df = df.resample('MS').last()
df.columns = df.columns.str.replace(r'\s+', ' ', regex=True).str.strip()
df.to_csv(f"{PATH_RAW}/bce_ims1.csv")
print("RRII updated")
print(df.index.max())

In [ ]:
df = pd.read_excel( df0 , sheet_name='IMS1.1', skiprows=6, skipfooter=13, header=[0,1,2])
df.columns = df.columns.map('{0[0]}_{0[1]}_{0[2]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()


df['year'] = df['periodo_unnamed: 0_level_1_unnamed: 0_level_2'].ffill()
months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }
df['month'] = df['periodo_unnamed: 1_level_1_unnamed: 1_level_2'].astype(str).str.lower().replace(months, regex=True)

df.dropna(subset='year', inplace=True)
df.year = df.year.astype(int)

df["date"] = pd.to_datetime(df[['year', 'month']].assign(DAY=1), format='%b')
df.dropna(subset='date', inplace=True)
df['date'] = pd.to_datetime( df['date'] )
df.columns = df.columns.str.replace(r'\(.*?\)', '', regex=True).str.strip()

df.columns = df.columns.str.replace(r'_unnamed: \d+_level_\d+', '', regex=True).str.strip()
df.columns = df.columns.str.replace(r'_', ' ', regex=True).str.strip()
df.set_index('date', inplace=True)

df.columns = df.columns.str.replace(r'\s+', ' ', regex=True).str.strip()

target = ["especies monetarias en circulacion", 
"moneda fraccionaria" ,
"dinero electronico" ,
"depositos a la vista", 
"oferta monetaria m1",
"cuasidinero",
"liquidez total m2",
"reservas bancarias",
"base monetaria bm",
"multiplicador m1/bm",
"multiplicador m2/bm"]

def rename_columns(df, target_list):
    new_columns = {}
    
    for column in df.columns:
        for target_str in target_list:
            if target_str in column:
                new_columns[column] = target_str
                break
    
    df.rename(columns=new_columns, inplace=True)
rename_columns(df, target)
df = df[target]

df.to_csv(f"{PATH_RAW}/bce_ims11.csv")
print("Oferta Monetaria updated")
print(df.index.max())

In [ ]:
df = pd.read_excel( df0 , sheet_name='IMS2', skiprows=6, skipfooter=2, header=[0,1])
df.columns = df.columns.map('{0[0]}_{0[1]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
df['variable'] = df[ [col for col in df.columns if "unnamed: 0" in col] ]
df['variable'] = df['variable'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()

# Remove parentheses
df['variable'] = df['variable'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
df.dropna(subset='variable', inplace=True)

df['variable'] = df['variable'].str.replace( r'[.\d]+' , '', regex=True).str.strip()
df['variable'] = df['variable'].str.replace(r'\s+', ' ', regex=True).str.strip()

df.drop([ col for col in df.columns if "unnamed" in col ], axis=1, inplace=True)

df = pd.melt(df, id_vars='variable', var_name='date', value_name='value')

months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }

df['date'] = df['date'].astype(str).str.lower().replace(months, regex=True)
df['date'] = df['date'].str.replace('_', '-')

def remove_repeated_year(date_str):
    # Match the format 'YYYY-YYYY-rest' or 'YYYY-rest'
    match = re.match(r'(\d{4})-(\d{4})-(.*)', date_str)
    if match and match.group(1) == match.group(2):
        return f"{match.group(1)}-{match.group(3)}"
    return date_str
df['date'] = df['date'].apply(remove_repeated_year)
df['date'] = df['date'].replace(r'^(\d{4}-\d{2})$', r'\1-01', regex=True)

def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S')
    except ValueError:
        try:
            return pd.to_datetime(date_str, format='%Y-%m-%d')
        except ValueError:
            return pd.NaT
df['date'] = df['date'].apply(parse_date)
df.date = pd.to_datetime(df.date)
df.set_index('date', inplace=True)

df = df.pivot(columns='variable', values='value')
df.rename(columns = {'ri' : 'reservas internacionales'} , inplace=True)

df.to_csv(f"{PATH_RAW}/bce_ims2.csv")
print("Monetary")
print(df.index.max())

**Captaciones (IMS13 & IMS7)**

In [ ]:
df = pd.read_excel( df0 , sheet_name='IMS13', skiprows=8, skipfooter=2, header=[0,1])
df.columns = df.columns.map('{0[0]}_{0[1]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
df['variable'] = df[ [col for col in df.columns if "unnamed: 0" in col] ]
df['variable'] = df['variable'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()

df['variable'] = df['variable'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
df.dropna(subset='variable', inplace=True)

df['variable'] = df['variable'].str.replace( r'[.\d]+' , '', regex=True).str.strip()
df['variable'] = df['variable'].str.replace(r'\s+', ' ', regex=True).str.strip()

df.drop([ col for col in df.columns if "unnamed" in col ], axis=1, inplace=True)

df = pd.melt(df, id_vars='variable', var_name='date', value_name='value')

months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }

df['date'] = df['date'].astype(str).str.lower().replace(months, regex=True)
df['date'] = df['date'].str.replace('_', '-')

def remove_repeated_year(date_str):
    # Match the format 'YYYY-YYYY-rest' or 'YYYY-rest'
    match = re.match(r'(\d{4})-(\d{4})-(.*)', date_str)
    if match and match.group(1) == match.group(2):
        return f"{match.group(1)}-{match.group(3)}"
    return date_str
df['date'] = df['date'].apply(remove_repeated_year)
df['date'] = df['date'].replace(r'^(\d{4}-\d{2})$', r'\1-01', regex=True)

def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S')
    except ValueError:
        try:
            return pd.to_datetime(date_str, format='%Y-%m-%d')
        except ValueError:
            return pd.NaT
df['date'] = df['date'].apply(parse_date)
df.date = pd.to_datetime(df.date)
df.set_index('date', inplace=True)

df.dropna(subset='value', inplace=True)
df = df[df['variable'].str.contains('depositos vista panorama') | df['variable'].str.contains('cuasidinero') ]

df = df.pivot(columns='variable', values='value')

df.rename( columns=lambda x: f"ims13_captaciones_{x.replace(' ', '_')}" , inplace=True)
df = pd.DataFrame(df.resample("MS").last()).sort_index()
df.to_csv(f"{PATH_RAW}/bce_ims13.csv")
print(f"Depositos updated: {df.index.max()}")

In [ ]:
df = pd.read_excel( df0 , sheet_name='IMS14', skiprows=8, skipfooter=2, header=[0,1])
df.columns = df.columns.map('{0[0]}_{0[1]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
df['variable'] = df[ [col for col in df.columns if "unnamed: 0" in col] ]
df['variable'] = df['variable'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()

df['variable'] = df['variable'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
df.dropna(subset='variable', inplace=True)

df['variable'] = df['variable'].str.replace( r'[.\d]+' , '', regex=True).str.strip()
df['variable'] = df['variable'].str.replace(r'\s+', ' ', regex=True).str.strip()

df.drop([ col for col in df.columns if "unnamed" in col ], axis=1, inplace=True)

modified_values = []
for i, row in df.iterrows():
    if "cartera" in row['variable'].lower():
        modified_value = f"{row['variable']} - {df.at[i-1, 'variable']}"
        modified_values.append(modified_value)
    else:
        modified_values.append(row['variable'])

df['variable'] = modified_values


df = pd.melt(df, id_vars='variable', var_name='date', value_name='value')

months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }

df['date'] = df['date'].astype(str).str.lower().replace(months, regex=True)
df['date'] = df['date'].str.replace('_', '-')


def remove_repeated_year(date_str):
    # Match the format 'YYYY-YYYY-rest' or 'YYYY-rest'
    match = re.match(r'(\d{4})-(\d{4})-(.*)', date_str)
    if match and match.group(1) == match.group(2):
        return f"{match.group(1)}-{match.group(3)}"
    return date_str
df['date'] = df['date'].apply(remove_repeated_year)
df['date'] = df['date'].replace(r'^(\d{4}-\d{2})$', r'\1-01', regex=True)

def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%Y-%m-%d %H:%M:%S')
    except ValueError:
        try:
            return pd.to_datetime(date_str, format='%Y-%m-%d')
        except ValueError:
            return pd.NaT
df['date'] = df['date'].apply(parse_date)
df.date = pd.to_datetime(df.date)
df.set_index('date', inplace=True)

df.dropna(subset='value', inplace=True)
df = df[df['variable'].str.contains('cartera') ]

df = df.pivot(columns='variable', values='value')
df.rename( columns=lambda x: f"{x.replace(' - ', '_')}" , inplace=True)

df = pd.DataFrame(df.resample("MS").last()).sort_index()
df.to_csv(f"{PATH_RAW}/bce_ims14.csv")
print(f"Cartera updated: {df.index.max()}")

**Balanza de Pagos**

In [ ]:
url = "https://contenido.bce.fin.ec/documentos/Estadisticas/SectorExterno/BalanzaPagos/indice.htm"

r = requests.get(url, headers=hdr, verify=False)
r = r.content

soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

links = [ link.get('href') for link in links if "htm" in link.get('href') ]
def get_boletin_number(url):
    return int(url.split('boletin')[-1].split('/')[0])
    
links = max(links, key=get_boletin_number)

r = requests.get(links, headers=hdr, verify=False)
r = r.content

soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

get = [ link.get('href') for link in links if "bp" in link.get('href').lower() ]

df = pd.ExcelFile( BytesIO(requests.get(get[0], headers=hdr, verify=False).content) )
df = pd.read_excel(df, skiprows=7, skipfooter=5)

df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
df.dropna(inplace=True)

df = df[ ( df['componentes normalizados'].str.contains('CUENTA') & ~df['componentes normalizados'].str.lower().str.contains('otras')) | df['componentes normalizados'].str.contains('ACTIVOS DE RESERVA') | df['componentes normalizados'].str.lower().str.contains('ingreso primario') | df['componentes normalizados'].str.lower().str.contains('ingreso secundario') | ( df['componentes normalizados'] == ('SERVICIOS')) | ( df['componentes normalizados'] == ('BIENES Y SERVICIOS')) ]
df['componentes normalizados'] = df['componentes normalizados'].astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
df = pd.melt(df, id_vars="componentes normalizados", var_name="date")
df = df[df['date'].str.contains("i")]

quarter_mapping = {
    'iv': 'Q4',
    'iii': 'Q3',
    'ii': 'Q2',
    'i': 'Q1',    
}

for roman, quarter in quarter_mapping.items():
    df['date'] = df['date'].str.replace(roman, quarter)
    
def quarter_to_date(quarter):
    q, year = quarter.split('-')
    month = {'Q1': '01', 'Q2': '04', 'Q3': '07', 'Q4': '10'}[q]
    return pd.to_datetime(f"{year}-{month}-01")

df['date'] = df['date'].apply(quarter_to_date)
df.set_index('date' , inplace=True)

df = pd.pivot(df, columns="componentes normalizados", values="value")
df.rename(columns = { 'bienes y servicios' : 'balanza comercial' } , inplace=True)
df.to_csv(f"{PATH_RAW}/bce_bp.csv")
print(f"BP updated: {df.index.max()}")

**Balanza Comercial**

In [ ]:
url = "https://contenido.bce.fin.ec/documentos/PublicacionesNotas/Catalogo/IEMensual/Indices/m2068062024.html"
r = requests.get(url, headers=hdr, verify=False)
r = r.content

soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

get = [ link.get('href') for link in links if "322-e" in link.get('href') ]

df = pd.read_excel( BytesIO(requests.get(get[0], headers=hdr, verify=False).content) , skiprows=7, header=[0,1], skipfooter=2)
df.columns = df.columns.map('{0[0]}_{0[1]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()

df.rename( columns=lambda x: "month" if "periodo" in x else x, inplace=True )
df = df[[ col for col in df.columns if "month" in col or col.startswith("exportaciones")  or col.startswith("importaciones") or col.startswith("balanza") ]]

df.rename( columns=lambda x: f"{x.replace('_', ' ')}" , inplace=True)

df.dropna(subset='month', inplace=True)
df.columns = df.columns.str.replace(r'\(.*?\)', '', regex=True).str.strip().str.replace('  ', ' ')
df = df[df['month'].str.isnumeric()==False]
df['year'] = df['month'].str.extract(r'(\d+)').ffill()
df = df[~df['month'].str.contains(r'\d')]
months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }

df['month'] = df['month'].astype(str).str.lower().replace(months, regex=True)
df["date"] = pd.to_datetime(df[['year', 'month']].assign(DAY=1), format='%b')

df = df[[ col for col in df.columns if "date" in col or col.startswith("exportaciones")  or col.startswith("importaciones") or col.startswith("balanza") ]]
df.set_index('date', inplace = True)
df.to_csv(f"{PATH_RAW}/bce_bc.csv")

print(f"Balanza Comercial updated: {df.index.max()}")

In [ ]:
get = [ link.get('href') for link in links if "312-e" in link.get('href') ]
df = pd.read_excel( BytesIO(requests.get(get[0], headers=hdr, verify=False).content) , skiprows=7, header=[0,1,2], skipfooter=2)
df.columns = df.columns.map('{0[0]}_{0[1]}_{0[2]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
df.rename( columns=lambda x: "year" if "periodo" in x and "0_" in x else x, inplace=True )
df.rename( columns=lambda x: "month" if "periodo" in x and "1_" in x else x, inplace=True )

df.dropna(subset='month', inplace=True)
df['year'] = df['year'].ffill().astype(int)
df = df[~df['month'].str.contains("-")]

df.columns = df.columns.str.replace(r'\(.*?\)', '', regex=True).str.strip().str.replace('  ', ' ')
months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }

df['month'] = df['month'].astype(str).str.lower().replace(months, regex=True)
df["date"] = pd.to_datetime(df[['year', 'month']].assign(DAY=1), format='%b')
df.drop(['year', 'month'], axis=1, inplace=True)
df.set_index('date', inplace=True)
df.columns = df.columns.astype('str').str.replace('_unnamed:',' ').str.replace(r'(\d+_level_\d+)',' ', regex=True)
df.columns = df.columns.astype('str').str.replace('_',' ').str.replace('  ',' ')
df.rename( columns=lambda x: f"exportaciones {x}" if "exportaciones" not in x else x , inplace = True )

df.to_csv(f"{PATH_RAW}/bce_exports.csv")
print(f"Exportaciones updated: {df.index.max()}")

In [ ]:
get = [ link.get('href') for link in links if "316-e" in link.get('href') ]

df = pd.read_excel( BytesIO(requests.get(get[0], headers=hdr, verify=False).content) , skiprows=7, header=[0,1], skipfooter=2)
df.columns = df.columns.map('{0[0]}_{0[1]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
df.rename( columns=lambda x: "year" if "periodo" in x and "0_" in x else x, inplace=True )
df.rename( columns=lambda x: "month" if "periodo" in x and "1_" in x else x, inplace=True )

df.dropna(subset='month', inplace=True)
df['year'] = df['year'].ffill().astype(int)
df = df[~df['month'].str.contains("-")]

df.columns = df.columns.str.replace(r'\(.*?\)', '', regex=True).str.strip().str.replace('  ', ' ')
months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }

df['month'] = df['month'].astype(str).str.lower().replace(months, regex=True)
df["date"] = pd.to_datetime(df[['year', 'month']].assign(DAY=1), format='%b')
df.drop(['year', 'month'], axis=1, inplace=True)
df.set_index('date', inplace=True)

df.columns = df.columns.astype('str').str.replace('_unnamed:',' ').str.replace(r'(\d+_level_\d+)',' ', regex=True)
df.columns = df.columns.astype('str').str.replace('_',' ').str.replace('  ',' ')
df.rename( columns=lambda x: f"importaciones {x}" if "importaciones" not in x else x , inplace = True )

df.to_csv(f"{PATH_RAW}/bce_imports.csv")
print(f"Importaciones updated: {df.index.max()}")

In [ ]:
get = [ link.get('href') for link in links if "341-e" in link.get('href') ]

df = pd.read_excel( BytesIO(requests.get(get[0], headers=hdr, verify=False).content) , skiprows=7, header=[0,1], skipfooter=2)
df.columns = df.columns.map('{0[0]}_{0[1]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
df.rename( columns=lambda x: "date" if "periodo" in x else x, inplace=True )
df.dropna(subset='date', inplace=True)

# Extract Year
df['year'] = df['date'].str.extract(r'(\d{4})')
df['year'] = df['year'].ffill()
df.dropna(subset='year', inplace=True)

df['month'] = df['date'].str.replace(r'\d{4} ', '', regex=True)
months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }

df['month'] = df['month'].astype(str).str.lower().replace(months, regex=True)
df["date"] = pd.to_datetime(df[['year', 'month']].assign(DAY=1), format='%b')
df.drop(['year', 'month'], axis=1, inplace=True)
df.set_index('date', inplace=True)

df.columns = df.columns.astype('str').str.replace(r'\(.*?\)', '', regex=True).str.strip()
df.columns = df.columns.astype('str').str.replace('_unnamed:',' ').str.replace(r'(\d+_level_\d+)',' ', regex=True)
df.columns = df.columns.astype('str').str.replace('_',' ').str.replace('  ',' ')
df.rename( columns=lambda x: f"tdc {x}" if "date" not in x else x , inplace = True )

df.to_csv(f"{PATH_RAW}/bce_tdc.csv")
print(f"Tipo de Cambio updated: {df.index.max()}")

In [ ]:
get = [ link.get('href') for link in links if "342-e" in link.get('href') ]

df = pd.read_excel( BytesIO(requests.get(get[0], headers=hdr, verify=False).content) , skiprows=7, header=[0,1], skipfooter=2)
df.columns = df.columns.map('{0[0]}_{0[1]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()

df.rename( columns=lambda x: "date" if "periodo" in x else x, inplace=True )
df.dropna(subset='date', inplace=True)

# Extract Year
df['year'] = df['date'].str.extract(r'(\d{4})')
df['year'] = df['year'].ffill()
df.dropna(subset='year', inplace=True)

df['month'] = df['date'].str.replace(r'\d{4} ', '', regex=True)
months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }

df['month'] = df['month'].astype(str).str.lower().replace(months, regex=True)
df["date"] = pd.to_datetime(df[['year', 'month']].assign(DAY=1), format='%b')
df.drop(['year', 'month'], axis=1, inplace=True)
df.set_index('date', inplace=True)

df.columns = df.columns.astype('str').str.replace('_unnamed:',' ').str.replace(r'(\d+_level_\d+)',' ', regex=True)
df.columns = df.columns.astype('str').str.replace('_',' ').str.replace('  ',' ')
df.columns = df.columns.astype('str').str.replace(r'\(\d?\)', '', regex=True).str.strip()
df.columns = df.columns.astype('str').str.replace(r'[()]', '', regex=True).str.strip()
df.rename( columns=lambda x: f"tdc_usd {x}" if "date" not in x else x , inplace = True )

df.to_csv(f"{PATH_RAW}/bce_tdc_usd.csv")
print(f"Tipo de Cambio updated: {df.index.max()}")

**Indice de Precios de la Construccion**

In [ ]:
url = "https://www.ecuadorencifras.gob.ec/indice-de-precios-de-la-construccion/"
r = requests.get(url=url, verify=False).content

soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

excel_links = [ link.get('href') for link in links if 
               link.get('href') is not None and
               "tab" in link.get('href').lower() and
               "excel" in link.get('href').lower() 
              ]

file = zipfile.ZipFile((io.BytesIO( urlopen(excel_links[0]).read() )))
f = [ f for f in file.namelist() if "xlsx" in f and "historica" in f and "general" in f]

if len(f)==1: 
    df = pd.ExcelFile(file.open(f[0]).read())
    sheets = [ '1. índice' ]
    dfi = pd.read_excel( df, sheet_name=sheets[0] , skiprows=4 )
    dfi.columns = dfi.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
    dfi = dfi[[ col for col in dfi.columns if "unnamed" not in col ]]
    dfi.rename(columns = {'anos' : 'year'} , inplace=True)
    dfi = pd.melt( dfi , id_vars = "year" , var_name = 'month' )

    months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
              'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }
    dfi['month'] = dfi['month'].astype(str).str.lower().replace(months, regex=True)

    dfi["date"] = pd.to_datetime(dfi[['year', 'month']].assign(DAY=1), format='%b')
    dfi = dfi[['value', 'date']]
    dfi.set_index('date', inplace=True)
    dfi.rename( columns = { 'value' : 'IPCO' } , inplace=True)
    dfi = dfi.dropna().sort_index()
    dfi.IPCO = dfi.IPCO.astype('str').str.replace(',','.')
    dfi['IPCO'] = pd.to_numeric(dfi['IPCO'].replace(r'[^0-9.]', '', regex=True))
    
    dfi.to_csv(f"{PATH_RAW}/bce_ipco.csv")
    print(f"Indice de Precios de la Construccion updated: {dfi.index.max()}")

**Indice de Situacion Presente**

In [ ]:
url = "https://contenido.bce.fin.ec/documentos/PublicacionesNotas/Catalogo/IEMensual/Indices/m2068062024.html"
r = requests.get(url=url, verify=False).content
soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

get = [ link.get('href') for link in links if link.get('href') is not None and "462-e" in link.get('href') ]

df = pd.read_excel( BytesIO(requests.get(get[0], headers=hdr, verify=False).content) , skiprows=7, header=[0,1], skipfooter=2)
df.columns = df.columns.map('{0[0]}_{0[1]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()

df.rename( columns=lambda x: "year" if "periodo" in x and "0_level" in x else x, inplace=True )
df.rename( columns=lambda x: "month" if "periodo" in x and "1_level" in x else x, inplace=True )

df.dropna(subset='month', inplace=True)
df['month'] = df['month'].astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()

df.loc[df[df['month'] == 'enero'].last_valid_index(), 'year']  = datetime.now().year
df['year'] = df['year'].ffill()
df.dropna(subset='month', inplace=True)
df['month'] = df['month'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }
df['month'] = df['month'].astype(str).str.lower().replace(months, regex=True)
df["date"] = pd.to_datetime(df[['year', 'month']].assign(DAY=1), format='%b')
df.set_index('date', inplace=True)


df.columns = df.columns.str.replace(r'\(.*?\)', '', regex=True).str.strip()
df.drop(['year', 'month'], axis=1, inplace=True)
df.columns = df.columns.astype(str).str.replace("_" , " ")

df.to_csv(f"{PATH_RAW}/bce_situacion.csv")
print(f"Indice de Situacion Presente: {df.index.max()}")

## Banco de Peru

In [ ]:
# get = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api/"
today = date.today().year + 1

variables = {
    "PD04712XD" : "bcrp_embig" ,
    "PN01134XM" : "bcrp_embigdiff" ,
}


df = pd.DataFrame()

months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
      'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','set': '09','oct': '10','nov': '11','dic': '12','dec': '12' }

for k, v in variables.items() :
    url = f"https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{k}/json/1980-01-01/{today}-01-01"
    resp = requests.get(url)
    data = resp.json()
    dfi = pd.DataFrame(data['periods'] )
    
    dfi.rename(columns = { 'name' : 'date', 'values' : v }, inplace=True)
    dfi[v] = pd.to_numeric(dfi[v].astype(str).str.extract('(\d+.\d+)', expand=False) )
    
    
    dfi['date'] = dfi['date'].astype(str).str.lower().replace(months, regex=True)
    dfi['date'] = dfi['date'].astype(str).str.replace(".", "-")
    if v == "bcrp_embigdiff" :
        dfi.date = pd.to_datetime(dfi.date, format = "%m-%Y")
    else :
        dfi.date = pd.to_datetime(dfi.date, format = "%d-%m-%y")
    dfi.set_index('date' , inplace=True)
    
    df = df.merge(dfi, left_index=True, right_index=True, how='outer')

df.to_csv(f"{PATH_RAW}/bcrp_embig.csv")
print(f"EMBIG: {df.index.max()}")

## Federal Reserve FRED

In [ ]:
series = ["DCOILWTICO", "WUIECU", "IMP3310", "EXP3310", "CPIAUCSL"]

api_key = "6082258bcc5fa73a032d9d60c890f744"

df = pd.DataFrame()

for serie in series :

    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={serie}&realtime_end=9999-12-31&api_key={api_key}&file_type=json"
    response = requests.get(url)
    response = response.json()
    dfi = pd.DataFrame(response['observations'])
    dfi = dfi[['value', 'date']]
    dfi.set_index('date', inplace=True)
    dfi.index = pd.to_datetime(dfi.index)
    dfi.rename(columns = { "value" : f"fred_{serie}" }, inplace=True)
    dfi[f"fred_{serie}"] = dfi[f"fred_{serie}"].apply(pd.to_numeric, errors='coerce')
    df = df.merge(dfi, left_index=True, right_index=True, how='outer')
    df = pd.DataFrame(df.resample("D").mean())
    #df = pd.concat([df, dfi], axis=1)
    df.sort_index(inplace=True)
    
df.to_csv(f"{PATH_RAW}/fred_data.csv")
print(f"FRED Updated: {df.index.max()}")

## Inflacion en paises seleccionados
- EEUU
- China
- Colombia
- Peru

In [ ]:
df = pd.DataFrame()

series = [ "CPIAUCSL" , "CHNCPIALLMINMEI" ]
api_key = "6082258bcc5fa73a032d9d60c890f744"

for serie in series :
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={serie}&observation_start=1990-01-01&realtime_start={datetime.today().strftime('%Y-%m-%d')}&realtime_end=9999-12-31&api_key={api_key}&file_type=json"
    response = requests.get(url)
    response = response.json()
    dfi = pd.DataFrame(response['observations'])
    dfi = dfi[['value', 'date']]
    dfi.rename(columns = { "value" : f"{serie}" }, inplace=True)
    dfi[serie] = pd.to_numeric(dfi[serie], errors='coerce')
    dfi.dropna(inplace=True)
    dfi.set_index('date', inplace=True)
    dfi.index = pd.to_datetime(dfi.index)
    dfi.sort_index(inplace=True)
    dfi = dfi[~dfi.index.duplicated(keep='first')]
    df = df.merge(dfi, left_index=True, right_index=True, how='outer')

df.rename(columns = { "CPIAUCSL" : f"CPI EEUU" , "CHNCPIALLMINMEI" : "CPI CN" }, inplace=True)
df.to_csv(f"{PATH_RAW}/international_data.csv")
print(f"Inflacion 1 Updated: {df.index.max()}")

In [ ]:
url = "https://www.dane.gov.co/index.php/estadisticas-por-tema/precios-y-costos/indice-de-precios-al-consumidor-ipc"
r = requests.get(url, headers=hdr, verify=False)
r = r.content

soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

get = [ link.get('href') for link in links if link.get('href') is not None and "Indices" in link.get('href') ]
get = f"https://www.dane.gov.co/{get[0]}"
dfi = pd.read_excel( get , skiprows=8, skipfooter=3)

dfi = pd.melt( dfi , id_vars = "Mes" , var_name = 'year' )
dfi.rename( columns = { "Mes" : "month" , "value" : "CPI CO" } , inplace=True)
dfi.dropna( subset= "CPI CO" , inplace = True)

months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }

dfi['month'] = dfi['month'].astype(str).str.lower().replace(months, regex=True)
dfi["date"] = pd.to_datetime(dfi[['year', 'month']].assign(DAY=1), format='%b')

dfi = dfi[[ 'CPI CO' , 'date' ]].set_index('date')
df = df.merge(dfi, left_index=True, right_index=True, how='outer')
df.to_csv(f"{PATH_RAW}/international_data.csv")
print(f"Inflacion 2 Updated: {df.index.max()}")

In [ ]:
# get = "https://estadisticas.bcrp.gob.pe/estadisticas/series/api/"
today = date.today().year + 1

url = f"https://estadisticas.bcrp.gob.pe/estadisticas/series/api/PN38705PM/json/1980-01-01/{today}-01-01"
resp = requests.get(url)
data = resp.json()
dfi = pd.DataFrame(data['periods'] )
dfi.rename(columns = { 'name' : 'date', 'values' : "CPI PE" }, inplace=True)
dfi['CPI PE'] = pd.to_numeric(dfi['CPI PE'].astype(str).str.extract('(\d+.\d+)', expand=False) )

dfi["date"] = dfi["date"].astype(str).str.lower().replace(months, regex=True)
dfi['date'] = dfi['date'].astype(str).str.replace(r'(\d{2}).(\d{4})', r"\2-\1-01", regex=True)
dfi.date = pd.to_datetime(dfi.date)
dfi.set_index('date' , inplace=True)

df = df.merge(dfi, left_index=True, right_index=True, how='outer')
df.to_csv(f"{PATH_RAW}/international_data.csv")
print(f"Inflacion 3 Updated: {df.index.max()}")

## Servicio de Rentas Internas

- VAT
- Ventas
- Vehiculos

**Estadisticas de Recaudacion**

In [ ]:
url = "https://www.sri.gob.ec/estadisticas-generales-de-recaudacion-sri"
r = requests.get(url, headers=hdr, verify=False)
r = r.content

soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }

get = [ link.get('href') for link in links if link.get('href') is not None and "xlsx" in link.get('href') and "recauda" in link.get('href').lower() and "estad" in link.get('href').lower()]
if len(get)>0 :
    get = get[0]
    year = re.findall(r'\d{4}', get.split("/")[-1])[0]
    df = pd.ExcelFile(get)
    df = pd.read_excel(
        df,
        sheet_name= [f for f in df.sheet_names if "abierta" in f.lower() ][0] ,
        skiprows=5 ,
        skipfooter=15 ,
    )

    df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
    df['conceptos'] = df['conceptos'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
    df['conceptos'] = df['conceptos'].str.replace(r'\(.*?\)', '', regex=True).str.replace('  ', ' ').str.strip()
    df.dropna(subset='conceptos', inplace=True)

    df = df[df.index<=df[df['conceptos'].str.startswith("otros")].index[0]]
    df['the_index'] = df['conceptos'].index.astype(str)
    df['conceptos'] = df['the_index'] + " sri " + df['conceptos']
    df.drop('the_index', axis=1, inplace=True)
    
    df = pd.melt(df, id_vars="conceptos", var_name="date").dropna(subset=[ 'conceptos', 'value'] )
 
    df['date'] = df['date'].astype(str).str.lower().replace(months, regex=True)
    df = df[df['date'].str.match(r'^\d+(\.\d+)?$')]
    #FIND YEAR
    df['date'] = f"{year}-"+df['date']+"-01"
    df["date"] = pd.to_datetime( df["date"] , format='%Y-%m-%d')
    df.set_index('date' , inplace=True)
    
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df.dropna(subset='value', inplace=True)
    df = df[df.value!=0]
    
    df = pd.pivot(df, columns="conceptos", values="value")
    
    df = df[ sorted(df.columns, key=lambda x: int(x.split()[0])) ]

    try :
        df0 = pd.read_csv(f"{PATH_RAW}/sri_recaudacion.csv", index_col='date', parse_dates=True)
        df0 = df0[ df0.index.year != df.index.year.max() ]
        df = pd.concat( [df0, df] , axis=0)
    except :
        df0 = df
    
    df.to_csv(f"{PATH_RAW}/sri_recaudacion.csv")
    print(f"Recaudacion SRI Updated: {df.index.max()}")

In [ ]:
df0 = pd.read_csv(f"{PATH_RAW}/sri_recaudacion.csv", index_col='date', parse_dates=True)

url = "https://www.sri.gob.ec/historico-estadisticas-generales-de-recaudacion"
r = requests.get(url, headers=hdr, verify=False)
r = r.content
soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')
get = [ link.get('href') for link in links if link.get('href') is not None and "xlsx" in link.get('href') and "recauda" in link.get('href').lower() and "estad" in link.get('href').lower()]
if 'df0' in locals() :
    get = [ g for g in get if int(re.findall(r'\d{4}', g.split("/")[-1].replace("%20",''))[0]) not in df0.index.year.unique() ]

column_names = { col.split(' ', 2)[-1]: col for col in df0.columns }
column_names['impuesto a la renta global'] = '1 sri impuesto a la renta recaudado'

for i in get :
    year = re.findall(r'\d{4}', i.split("/")[-1].replace("%20",''))[0]
    if int(year) in df0.index.year.unique() : continue
    print(f"Updating: {year}")

    if "xlsx" in i :
        df = pd.ExcelFile(i)
    elif "zip" in i :
        file = zipfile.ZipFile((io.BytesIO(urlopen(get[0]).read())))
        f = [s for s in file.namelist() if "estad" in s.lower() and "recaudac" in s.lower() ]
        df = pd.ExcelFile( file.open(f[0]).read() )
        
    try :
        if int(year)==2000 : open = year
        else : open = [f for f in df.sheet_names if "abierta" in f.lower() or "abierto" in f.lower() or "recaudac" in f.lower() ][0]
        df = pd.read_excel(
            df,
            sheet_name= open ,
            skiprows=5 ,
            skipfooter=15 ,
        )
    except :
        print(f"Not updated: {year}.")
        continue

    df.columns = df.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
    df['conceptos'] = df['conceptos'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
    df['conceptos'] = df['conceptos'].str.replace(r'\(.*?\)', '', regex=True).str.replace('  ', ' ').str.strip()
    df.dropna(subset='conceptos', inplace=True)

    try :
        df = df[df.index<=df[df['conceptos'].str.startswith("otros")].index[0]]
    except :
        None
    df['conceptos'] = df['conceptos'].map(column_names)
    
    df = pd.melt(df, id_vars="conceptos", var_name="date").dropna(subset=[ 'conceptos', 'value'] )
 
    df['date'] = df['date'].astype(str).str.lower().replace(months, regex=True)
    df = df[df['date'].str.match(r'^\d+(\.\d+)?$')]
    #FIND YEAR
    df['date'] = f"{year}-"+df['date']+"-01"
    df["date"] = pd.to_datetime( df["date"] , format='%Y-%m-%d')
    df.set_index('date' , inplace=True)
    
    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df.dropna(subset='value', inplace=True)
    df = df[df.value!=0]
    
    df = pd.pivot(df, columns="conceptos", values="value")
    
    df = df[ sorted(df.columns, key=lambda x: int(x.split()[0])) ]
    df0 = pd.concat( [df0, df] , axis=0)

df0.sort_index(inplace=True)
df0.to_csv(f"{PATH_RAW}/sri_recaudacion.csv")
print(f"Recaudacion SRI Updated: {df0.index.max()}")

**SRI**

- VAT
- Ventas
- Vehiculos

In [ ]:
# ========================================
# SRI website. Open

url = "https://www.sri.gob.ec/datasets"
r = requests.get(url, headers=hdr, verify=False)
r = r.content

**Ventas y Compras**

In [ ]:
try :
    df = pd.read_csv(f'{PATH_RAW}/sri_ventas.csv', index_col='date', parse_dates=True)
except :
    df = pd.DataFrame()


d = []
for i in re.findall('href=[\'"]?([^\'">]+[\_0-9].csv)', str(r)) :
    if "ventas" in i and int(i[-8:-4]) >= pd.to_datetime(df.index.max()).year :
        d.append(i)

for i in d :
    dfi = pd.read_csv( i , sep="|", encoding = 'latin1', decimal=',')
    dfi.columns = dfi.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
    dfi = dfi.rename( columns = {
            "ano" : "year",
            "anio" : "year",
            "mes" : "month",
            })
    dfi = dfi.loc[:, ~dfi.columns.str.contains('^unnamed')]
    dfi.drop(['codigo_sector_n1', 'provincia', 'canton'], axis=1, inplace=True)
    dfi["date"] = pd.to_datetime(dfi[['year', 'month']].assign(DAY=1), format='%b')
    dfi.drop(['year', 'month'], axis=1, inplace=True)
    dfi = dfi.fillna(0).groupby(['date']).apply(lambda x : x.astype(int).sum()).reset_index()
    dfi.set_index('date', inplace=True)
    
    if dfi.index.max() > df.index.max() :
        dfi = dfi[dfi.index>df.index.max()]
        df = pd.concat([df,dfi], axis=0)
        df.sort_index(inplace=True)
        print(f"Updated: {df.index.max()}")
    else :
        print("Data has not been updated.")
        continue

df.to_csv(f'{PATH_RAW}/sri_ventas.csv')

**Vehiculos**

In [ ]:
try :
    df0 = pd.read_csv(f"{PATH_RAW}/sri_carsales.csv", index_col="date", parse_dates=True)
    print(f"Last obs: {df0.index.max()}")
except :
    df0 = pd.DataFrame()

d = []
for i in re.findall('href=[\'"]?([^\'">]+[\_0-9].csv)', str(r)) :
    if "vehiculos" in i.lower() and ( int(i[-8:-4]) >= df0.sri_file.max() ) :
        d.append(i)

TRANSF = {
    "codigo vehiculo 1": "code",
    "sub categoria 1": "subcat_1",
    "marca": "maker",
    "modelo" : "model",
    "pais" : "country",
    "ano modelo": "car_year",
    "tipo" : "type",
    "clase" : "class",
    "sub clase" : "class_sub",
    "cilindraje" : "cylinder",
    "tipo combustible": "fuel",
    "tipo servicio": "service",
    "codigo color 1" : "color_code1",
    "color 1" : "color_code1",
    "codigo color 2" : "color_code2",
    "color 2" : "color_code2",
    "forma de adquisicion": "buy",
    "forma adquisicion": "buy",
    "tipo transaccion" : "buy",
    "codigo canton": "canton_code",
    "descripcion canton": "canton",
    "mes adquisicion" : "month_buy",
    "mes registro venta": "month_registry",
    "valor avaluo": "value",
    "fecha proceso" : "month_registry",
    "mes  registro venta": "month_registry",
    "mes registro venta": "month_registry",
    "fecha compra" : "month_buy",
    "avaluo": "value",
    "codigo sub categoria" : "subcat_1" ,
    "codigo vehiculo" : "code",
    "categoria" : "subcat_1",
    "persona natural - juridica" : "buyer_type"
}

xls = pd.ExcelFile(f'{PATH_RAW}/sri_dictionary.xlsx')


for u in d :
    dfi = pd.read_csv( u , sep=";", encoding = 'latin1')
    
    dfi.columns = dfi.columns.str.lower()
    dfi.columns = dfi.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    dfi.columns = dfi.columns.str.replace(" de ", " ")
    dfi.columns = dfi.columns.str.replace(r"\(.*\)","", regex=True)
    dfi.columns = dfi.columns.str.strip()
    dfi = dfi.rename( columns = TRANSF )
    dfi = dfi.loc[:, ~dfi.columns.str.contains('^unnamed')]

    dfi.dropna(how='all', inplace=True)
    
    dfi['sri_file'] = pd.to_datetime(u[-8:-4]).year
    df0 = df0[df0['sri_file']!=dfi.sri_file.max()]

    months = {'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12'}
    dfi['month_buy'] = pd.to_datetime(dfi['month_buy'].astype(str).str.lower().replace(months, regex=True))
    dfi['month_registry'] = dfi['month_registry'].astype(str).str.lower().replace(months, regex=True)
    try :
        dfi['month_registry'] = pd.to_datetime(dfi['month_registry'], format = '%d/%m/%Y')
    except :
        dfi['month_registry'] = pd.to_datetime(dfi['month_registry'])
        dfi["month_registry"] = dfi["month_registry"].dt.date
    
    dfi["date"] = pd.to_datetime(dfi["month_registry"])
    dfi.set_index("date", inplace=True)

    dfi.drop(['canton', 'service'], axis=1, inplace=True)

    dfi['value'] = dfi['value'].str.replace(',', '.').astype(float)

    for sheet in xls.sheet_names :
        if sheet not in dfi.columns : continue

        sheet_df = pd.read_excel( f'{PATH_RAW}/sri_dictionary.xlsx' , sheet_name=sheet)
        dfi[sheet] = dfi[sheet].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
        dfi[sheet] = dfi[sheet].str.replace(r'\.\.', '.', regex=True)
        dfi[sheet] = dfi[sheet].str.replace('  ', ' ')

        mapping_dict = dict(zip(sheet_df[sheet_df.columns[0]], sheet_df[sheet_df.columns[1]]))

        # unmatched entries
        unmatched_texts = dfi[sheet_df.columns[0]][~dfi[sheet_df.columns[0]].isin(mapping_dict)].unique()
        max_code = max(mapping_dict.values()) if mapping_dict else 0
        new_entries = {text: max_code + i + 1 for i, text in enumerate(unmatched_texts)}
        mapping_dict.update(new_entries)

        dfi[sheet_df.columns[0]] = dfi[sheet_df.columns[0]].map(mapping_dict)

        dfi[sheet] = pd.to_numeric(dfi[sheet], errors='coerce').astype('Int64')
        dfi[sheet] = dfi[sheet].apply(lambda x: '' if pd.isna(x) else x)

        updated_sheet_df = pd.DataFrame(list(mapping_dict.items()), columns=[sheet_df.columns[0], sheet_df.columns[1]])
        with pd.ExcelWriter( f'{PATH_RAW}/sri_dictionary.xlsx' , mode='a', if_sheet_exists='replace') as writer:
            updated_sheet_df.to_excel(writer, sheet_name=sheet, index=False)

    for i in [ 'subcat_1', 'code', 'car_year' ] :
        dfi[i] = pd.to_numeric(dfi[i], errors='coerce').astype('Int64')
        dfi[i] = dfi[i].apply(lambda x: '' if pd.isna(x) else x)

    dfi['month_registry'] = pd.to_datetime( dfi['month_registry'] , format='mixed' )
    dfi['month_buy'] = pd.to_datetime( dfi['month_buy'] , format='mixed' )
    
    df0 = pd.concat( [df0,dfi] , axis=0)
    print(f"Updated obs: {df0.index.max()}")
    
    df0.to_csv(f"{PATH_RAW}/sri_carsales.csv")


df = pd.read_csv(f"{PATH_RAW}/sri_carsales.csv", parse_dates=True, index_col='date')
df = pd.DataFrame( df.resample('MS').size() , columns=['total car sales'] )
df.to_csv(f"{PATH_RAW}/sri_totalcar.csv")

**IAENP**

In [ ]:
try :
    df0 = pd.read_csv(f"{PATH_RAW}/sri_iaenp.csv", index_col="date", parse_dates=True)
except :
    df0 = pd.DataFrame()
    
d = []
for i in re.findall('href=[\'"]?([^\'">]+.csv)', str(r)) :
    if "iaenp" in i.lower() and "mensual" in i.lower() :
        d.append(i)


if len(d)>0 :
    dfi = pd.read_csv( d[0] , sep=";", encoding = 'latin1', decimal=',')

    dfi.columns = dfi.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
    dfi.rename(columns = { 'ano' : 'year' , 'mes' : 'month'} , inplace=True)
    dfi["date"] = pd.to_datetime(dfi[['year', 'month']].assign(DAY=1), format='%b')
    dfi.set_index('date', inplace=True)
    dfi = dfi.filter(regex='^indice')

    for col in dfi.columns :
        if "actividad empresarial no petrolera" in col :
            dfi.rename(columns = {col : 'iaenp'}, inplace=True)

    if pd.isnull(df0.index.max()) :
        dfi.sort_index(inplace=True)
        print(f"Updated: {dfi.index.max()}")
    elif dfi.index.max() > df0.index.max() :
        dfi = dfi[dfi.index>df0.index.max()]
        dfi = pd.concat([df0,dfi], axis=0)
        dfi.sort_index(inplace=True)
        print(f"Updated: {dfi.index.max()}")
    else :
        print("Data has not been updated.")
else :
    print("Data not found.")

dfi.to_csv(f"{PATH_RAW}/sri_iaenp.csv")

**VAT -- Impuestos**

In [ ]:
df0 = pd.read_csv(f"{PATH_RAW}/sri_vat.csv", index_col=["date"],
                   parse_dates=True , dtype={
                        "value" : 'float' 
                   }
                  )

d = []
for i in re.findall('href=[\'"]?([^\'">]+[\_0-9].csv)', str(r)) :
    if "recaudacion" in i and ( ( int(i[-8:-4]) >= df0.index.year.max() ) or ( int(i[-8:-4]) == df0.index.year.max() and df0.index.month.max() < 12 ) ) :
        d.append(i)

d

In [ ]:
def clean_tax_type(tax_type):
    patterns = [
        (r'\bimpuesto\b', 'imp.'),
        (r'\bimp\b', 'imp.'),
        (r'\bcon\w*', 'contribuciones'),
        (r'\bamb\w*', 'ambiental'),
        (r'\bmin\w*', 'mineras'),
        (r'\bvehic\w*', 'vehiculos motorizados'),
        (r'\bren\w*', 'ing extraord. recur. no reno')
    ]
    for pattern, replacement in patterns:
        tax_type = re.sub(pattern, replacement, tax_type, flags=re.IGNORECASE)

    return tax_type.strip()

for i in d :

    dfi = pd.read_csv( i , sep="|", encoding = 'latin1', decimal=',')

    dfi.columns= dfi.columns.str.lower()
    dfi.rename( columns = {
        "anio": "year",
        "mes": "month",
        "grupo_impuesto": "tax_type",
        "subgrupo_impuesto" : "tax_subtype" ,
        "impuesto" : "tax" ,
        "gran_contribuyente" : "contributor_large" ,
        "sector_n1" : "sector" ,
        "tipo_contribuyente" : "contributor_type" ,
        "valor_recaudado" : "value"
        }, inplace=True)

    for col in dfi.columns :
        if "anio" in col : dfi.rename( columns = { col : "year" } , inplace=True )
    dfi.month = dfi.month.str.extract('(\d+)')
    dfi["date"] = pd.to_datetime(dfi[['year', 'month']].assign(DAY=1))
    dfi.drop(['year', 'month'], axis=1, inplace=True)

    dfi = dfi.loc[dfi["date"]>df0.index.max()]
    dfi.set_index("date", inplace=True)


    dfi.drop(['codigo_opera_familia'], axis=1, inplace=True)

    cols = ['tax_type', 
            'tax_subtype',
            'tax',
            'contributor_large',
            'contributor_type',
            'provincia',
            'canton'
           ]
    for col in cols :
        dfi[col] = dfi[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()

    dfi['tax_type'] = dfi['tax_type'].apply(clean_tax_type)
    dfi['tax_type'] = dfi['tax_type'].str.replace(r'\.\.', '.', regex=True)
    dfi['tax_type'] = dfi['tax_type'].apply(lambda text: re.sub(r'\b(ing extraord. recur. no reno)\b', '', text, flags=re.IGNORECASE).strip())
    dfi['tax_type'] = dfi['tax_type'].apply(lambda text: re.sub(r'(?<=\w)\.(?!\s|$)', '. ', text))
    dfi['tax_type'] = dfi['tax_type'].str.replace('motorizados motorizados', 'motorizados')

    dfi['tax_subtype'] = dfi['tax_subtype'].apply(clean_tax_type)
    dfi['tax_subtype'] = dfi['tax_subtype'].str.replace(r'\.\.', '.', regex=True)
    dfi['tax_subtype'] = dfi['tax_subtype'].apply(lambda text: re.sub(r'\([^)]*\)', '', text))
    dfi['tax_subtype'] = dfi['tax_subtype'].str.replace(r'imp. ambiental a la contribuciones vehiculos motorizados', 'imp. ambiental a la contaminacion vehicular', regex=True)
    dfi['tax_subtype'] = dfi['tax_subtype'].apply(lambda text: re.sub(r'(?<=\w)\.(?!\s|$)', '. ', text))
    dfi['tax_subtype'] = dfi['tax_subtype'].apply(lambda text: re.sub('ing extraord. recur. no reno operaciones internas', 'imp. ing. extraord. recur. no renovables', text))

    dfi['tax'] = dfi['tax'].apply(clean_tax_type)
    dfi['tax'] = dfi['tax'].str.replace(r'\.\.', '.', regex=True)
    dfi['tax'] = dfi['tax'].apply(lambda text: re.sub(r'\([^)]*\)', '', text))
    dfi['tax'] = dfi['tax'].apply(lambda text: re.sub(r'(?<=\w)\.(?!\s|$)', '. ', text))

    xls = pd.ExcelFile(f'{PATH_RAW}/sri_dictionary.xlsx')

    for sheet in xls.sheet_names :
        if sheet not in dfi.columns : continue

        sheet_df = pd.read_excel( f'{PATH_RAW}/sri_dictionary.xlsx' , sheet_name=sheet)

        mapping_dict = dict(zip(sheet_df[sheet_df.columns[0]], sheet_df[sheet_df.columns[1]]))

        # unmatched entries
        unmatched_texts = dfi[sheet_df.columns[0]][~dfi[sheet_df.columns[0]].isin(mapping_dict)].unique()
        max_code = max(mapping_dict.values()) if mapping_dict else 0
        new_entries = {text: max_code + i + 1 for i, text in enumerate(unmatched_texts)}
        mapping_dict.update(new_entries)

        dfi[sheet_df.columns[0]] = dfi[sheet_df.columns[0]].map(mapping_dict)

        dfi[sheet] = pd.to_numeric(dfi[sheet], errors='coerce').astype('Int64')
        dfi[sheet] = dfi[sheet].apply(lambda x: '' if pd.isna(x) else x)

        updated_sheet_df = pd.DataFrame(list(mapping_dict.items()), columns=[sheet_df.columns[0], sheet_df.columns[1]])
        with pd.ExcelWriter( f'{PATH_RAW}/sri_dictionary.xlsx' , mode='a', if_sheet_exists='replace') as writer:
            updated_sheet_df.to_excel(writer, sheet_name=sheet, index=False)


    dfi.rename(columns = {element: f'{element}_code' for i, element in enumerate(xls.sheet_names)}, inplace=True)
    df0 = pd.concat([df0, dfi], axis=0)

print(f"Last observation: {df0.index.max()}")
df0.to_csv(f"{PATH_RAW}/sri_vat.csv")

df0 = df0.resample('MS').agg(sri_vat=('value', 'sum'), sri_transactions=('value', 'size'))
df0.to_csv(f"{PATH_RAW}/sri_vat_timeseries.csv")

## CENACE

In [ ]:
# ===============================
# ===============================
# Electricity Generation (cenace)
url = 'http://www.cenace.gob.ec/wp-content/plugins/ez-addons/data/indicadores.xlsx'
df0 = pd.read_excel(requests.get(url=url, verify=False).content, header=0, sheet_name="datos")
df0.columns = df0.columns.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12'}
df0['mes'] = df0['mes'].astype(str).str.lower().replace(months, regex=True)
df0 = df0.rename(columns={"ano": "year" , "mes" : "month"})
df0["date"] = pd.to_datetime(df0[["year","month"]].assign(DAY=1))
df0 = df0.drop(["year","month"], axis=1)
df0.set_index("date", inplace=True)

df0.rename( columns=lambda x: f"cenace_{x}" , inplace=True) 

df0.to_csv(f'{PATH_RAW}/cenace_power.csv')
print("Electricity updated")
print(df0.index.max())

## Ecuador en Cifras

**Indices de Precios**

In [ ]:
# ========================================
# ========================================
# Ecuador en Cifras
# ========================================
# Indice de Precios al Productor
# https://www.ecuadorencifras.gob.ec/indice-de-precios-al-productor-de-disponibilidad-nacional/

url = "https://www.ecuadorencifras.gob.ec/indice-de-precios-al-productor-de-disponibilidad-nacional/"
r = requests.get(url, headers=hdr, verify=False)

soup = BeautifulSoup(r.content, 'html.parser')
links = soup.find_all('a')

links = [ link.get('href') for link in links if link.get('href') is not None and
               "excel" in link.get('href').lower() ]

if len(links) > 0 : get = links[0]
else : get = None

if get != None :    
    file = zipfile.ZipFile((io.BytesIO(urlopen(get).read())))
    f = [s for s in file.namelist() if "generales" in s]
    if len(f) == 1 :
        df = pd.read_excel(file.open(f[0]).read(), sheet_name="1. Indice_General")
    else :
        for i in f :
            try : df = pd.read_excel(file.open(i).read(), sheet_name="1. Indice_General")
            except: continue
    df["Unnamed: 0"] = df["Unnamed: 0"].astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
    df.columns = df.loc[df["Unnamed: 0"]=="anos"].iloc[0]
    df = df[df["anos"].str.isnumeric()==True]
    df = pd.melt(df, id_vars="anos", var_name="month")
    df = df[df["month"].isnull()==False]
    months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12'}
    df['month'] = df['month'].astype(str).str.lower().replace(months, regex=True)
    df.rename(columns={"anos": "year", "value": "ipp"}, inplace=True)
    df["date"] = pd.to_datetime(df[["year","month"]].assign(DAY=1))
    df.drop(['year', "month"], axis=1, inplace=True)
    df.set_index("date", drop=True, inplace=True)
    df = df[df['ipp'].notna()]
    df.to_csv(f"{PATH_RAW}/ine_ipp.csv")
    print(f"IPP Updated: {df.index.max()}")

In [ ]:
# ========================================
# Consumer Price Index
# Requires xlrd

months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12'}
url = "https://www.ecuadorencifras.gob.ec/indice-de-precios-al-consumidor/"

r = requests.get(url, headers=hdr, verify=False)
r = r.content

soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

links = [ link.get('href') for link in links if link.get('href') is not None and
               "excel" in link.get('href').lower() ]

if len(links) > 0 : get = links[0]
else : get = None

if get != None :
    print("Downloading CPI")
    file = zipfile.ZipFile((io.BytesIO(urlopen(get.replace(" ", "%20")).read())))
    f = [s for s in file.namelist() if "serie historica" in s.lower()]
    if len(f) == 1 :
        df = pd.read_excel(file.open(f[0]).read(), sheet_name=1)
        df["Unnamed: 0"] = df["Unnamed: 0"].astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
        df.columns = df.loc[df["Unnamed: 0"]=="meses"].iloc[0]
        df = df[df["meses"].str.isnumeric()==True]
        df = pd.melt(df, id_vars="meses", var_name="month")
        df['month'] = df['month'].astype(str).str.lower().replace(months, regex=True)
        df.rename(columns={"meses": "year", "value": "cpi"}, inplace=True)
        df["date"] = pd.to_datetime(df[["year","month"]].assign(DAY=1))
        df.drop(['year', "month"], axis=1, inplace=True)
        df.set_index("date", drop=True, inplace=True)
        df = df[df['cpi'].notna()]
        df.sort_index(inplace=True)
        df.to_csv(f"{PATH_RAW}/ine_cpi.csv")
        print(f"CPI Updated: {df.index.max()}")
    else : print(f"CPI Not Updated")
else : print(f"CPI Not Found")


Estadisticas laborales

In [ ]:
url = "https://www.ecuadorencifras.gob.ec/estadisticas-laborales-enemdu/"

r = requests.get(url=url, verify=False).content

soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

excel_links = [ link.get('href') for link in links if
               "xls" in link.get('href').lower() 
               and "tabulados" in link.get('href').lower() 
               and "laboral" in link.get('href').lower()
              ]

df = pd.read_excel(excel_links[0], sheet_name = "1. Poblaciones", skiprows=1, header=[0,1])
df.columns = df.columns.map('{0[0]}_{0[1]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
for col in df.columns :
    if "periodo" in col : df.rename(columns = {col : 'date'} , inplace=True)
    elif "indicadores" in col : df.rename(columns = {col : 'indicadores'} , inplace=True)
    elif "total" in col : df.rename(columns = {col : 'total'} , inplace=True)

df.indicadores = df.indicadores.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
df = df[['date', 'indicadores', 'total']]

months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
          'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }
df['date'] = "01-" + df['date'].astype(str).str.lower().replace(months, regex=True)

df.date = pd.to_datetime(df.date , format= "%d-%m-%y")
df.set_index('date', inplace=True)

target = ['poblacion total', 'poblacion en edad de trabajar (pet)', 'poblacion economicamente activa', 'empleo', 'empleo adecuado/pleno', 'subempleo', 'desempleo' ]
df = df[df['indicadores'].isin(target)]

df = pd.pivot(df, columns="indicadores", values="total")

df.rename( columns=lambda x: f"enemdu_{x}" , inplace=True) 

print(f"Updated: {df.index.max()}")
df.to_csv(f'{PATH_RAW}/ine_labor.csv')

In [ ]:
url = "https://www.ecuadorencifras.gob.ec/enemdu-trimestral/"

r = requests.get(url=url, verify=False).content

soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

excel_links = [ link.get('href') for link in links if
               "xls" in link.get('href').lower() 
               and "tabulados" in link.get('href').lower() 
               and "laboral" in link.get('href').lower()
              ]
df = pd.read_excel(excel_links[0], sheet_name = "1. Poblaciones", skiprows=1, header=[0,1])
df.columns = df.columns.map('{0[0]}_{0[1]}'.format)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
for col in df.columns :
    if "trimestre" in col : df.rename(columns = {col : 'date'} , inplace=True)
    elif "indicadores" in col : df.rename(columns = {col : 'indicadores'} , inplace=True)
    elif "total" in col : df.rename(columns = {col : 'total'} , inplace=True)

df.indicadores = df.indicadores.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower().str.strip()
df = df[['date', 'indicadores', 'total']]

months = {'IV' : '10' , 'III' : '07' , 'II' : '04' , 'I' : "01"}
df['date'] = df['date'].astype(str).replace(months, regex=True)

df.date = "01-" + df.date.str.replace(' ', '', regex=True)

df.date = pd.to_datetime(df.date , format= "%d-%m-%Y")
df.set_index('date', inplace=True)

target = [ 'poblacion economicamente activa' ]
df = df[df['indicadores'].isin(target)]

df = pd.pivot(df, columns="indicadores", values="total")

df.rename( columns=lambda x: f"enemdu_{x} trimestral" , inplace=True) 

print(f"Updated: {df.index.max()}")
df.to_csv(f'{PATH_RAW}/ine_pea quarter.csv')

Pobreza

- Incidencia de pobreza por ingreso
- Incidencia de pobreza extrema por ingreso
- Coeficiente de Gini

In [ ]:
url = "https://www.ecuadorencifras.gob.ec/pobreza-por-ingresos/"

r = requests.get(url=url, verify=False).content

soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

excel_links = [ link.get('href') for link in links if 
               link.get('href') is not None and
               "tabulados" in link.get('href').lower() 
              ]
file = zipfile.ZipFile((io.BytesIO( urlopen(excel_links[0]).read() )))
f = [ f for f in file.namelist() if "xlsx" in f]

if len(f)==1: 
    df = pd.ExcelFile(file.open(f[0]).read())
    
    sheets = [ '1.1.1.pobre_nacional' , '1.2.1.extpob_nacional', '2.1. Desigualdad_nacional ' ]
    df0 = pd.DataFrame()
    
    for sheet in sheets :
        dfi = pd.read_excel(df, sheet_name=sheet, skiprows=8)
        dfi.columns = dfi.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
        
        if sheet == "1.1.1.pobre_nacional" :
            COL = 'incidencia (1)'
            new_col = 'pobreza'
        elif sheet == "1.2.1.extpob_nacional" :
            COL = 'incidencia (1)'
            new_col = 'pobreza_extrema'
        elif sheet == "2.1. Desigualdad_nacional " :
            COL = 'indice de gini'
            new_col = 'gini_index'
        
        dfi.rename(columns = { 'periodo' : 'month' , 'unnamed: 2' : 'year', COL : new_col } , inplace=True)
        dfi['month'] = dfi['month'].ffill()
        dfi = dfi[['month', 'year', new_col]]
        
        months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
                  'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }
        dfi['month'] = dfi['month'].astype(str).str.lower().replace(months, regex=True)
        dfi[new_col] = pd.to_numeric(dfi[new_col], errors='coerce')
        dfi.dropna(subset= new_col , inplace=True)
        dfi["date"] = pd.to_datetime(dfi[['year', 'month']].assign(DAY=1), format='%b')
        
        dfi = dfi.set_index('date')[[new_col]]
        
        dfi.sort_index(inplace=True)
        df0 = pd.concat([df0, dfi], axis=1)
    
    df0.rename( columns=lambda x: f"enemdu_{x}" , inplace=True) 
    print(f"Updated: {df0.index.max()}")
    df0.to_csv(f'{PATH_RAW}/ine_pobreza.csv')


## FINANZAS

(Works on selenium +4.6)

```
conda install selenium==4.6.0
conda install selenium==4.25.0
```
Chrome driver: https://googlechromelabs.github.io/chrome-for-testing/#stable

In [ ]:
url = "https://www.finanzas.gob.ec/estadistica-nueva-metodologia-2013-2022/"
headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3' }

r = requests.get(url=url, verify=False, headers=headers).content
soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

links = [ link for link in links if link.get('data-type') is not None and "Operaciones de Ingresos" in link.contents[0] ]

max_id = -1
latest_link = None

for link in links:
    href = link['href']
    
    id_str = href.split('=')[-1]
    id_int = int(id_str)
    # Compare and store the link with the highest ID
    if id_int > max_id:
        max_id = id_int
        latest_link = href

In [ ]:
from selenium import webdriver
import shutil
import time
from selenium.webdriver.chrome.service import Service
download_folder = "C:/Users/guerr/Downloads/"

#chromeOptions = webdriver.ChromeOptions()
#prefs = {"download.default_directory" : r"C:\\Users\\guerr\\Dropbox\\20240501 BID Nowcasting LATAM\\ecuador\\raw"}
#chromeOptions.add_experimental_option("prefs", prefs)
#driver = webdriver.Chrome(options=chromeOptions)
driver = webdriver.Chrome()
driver.get(latest_link)
time.sleep(10)
driver.quit()


target = [ f for f in os.listdir(download_folder) if "SPNF" in f]
if len(target) > 0 :
    target = target[0]
    shutil.move(f"{download_folder}/{target}", f"{PATH_RAW}/{target}")

dfi = pd.read_excel(f"{PATH_RAW}/{target}" , sheet_name="SPNF", skiprows=5, header = [0,1])
dfi.columns = dfi.columns.map('{0[0]}_{0[1]}'.format)
dfi.columns = dfi.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()

t = [ "codigo" , "transacciones" , "mensual" ]
dfi = dfi[[col for col in dfi.columns if any(target in col for target in t)]]

for col in dfi :
    if "codigo" in col : dfi.rename( columns = { col : 'codigo' } , inplace=True)
    elif "transacciones" in col : dfi.rename( columns = { col : 'transacciones' } , inplace=True)
dfi.columns = dfi.columns.str.replace('mensual_', '', regex=False)

dfi.codigo = pd.to_numeric(dfi['codigo'], errors='coerce')
dfi = dfi[(dfi['codigo'] < 100)]

dfi.drop('codigo', axis=1, inplace=True)

dfi = pd.melt(dfi, id_vars="transacciones", var_name="date")
dfi.date = pd.to_datetime(dfi.date)
dfi.transacciones = dfi.transacciones.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
dfi['transacciones'] = dfi['transacciones'].str.replace(r'\(.*?\)', '', regex=True).str.lower().str.strip().str.replace(" ", "_")

dfi.set_index('date', inplace=True)
dfi = pd.pivot(dfi, columns="transacciones", values="value")

dfi.rename( columns=lambda x: f"spnf_{x}" , inplace=True) 
print(f"Updated: {dfi.index.max()}")
dfi.to_csv(f'{PATH_RAW}/mef_spnf.csv')

In [ ]:
dfi = pd.read_excel(f"{PATH_RAW}/{target}" , sheet_name="GC", skiprows=5, header = [0,1])
dfi.columns = dfi.columns.map('{0[0]}_{0[1]}'.format)
dfi.columns = dfi.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()

t = [ "codigo" , "transacciones" , "mensual" ]
dfi = dfi[[col for col in dfi.columns if any(target in col for target in t)]]

for col in dfi :
    if "codigo" in col : dfi.rename( columns = { col : 'codigo' } , inplace=True)
    elif "transacciones" in col : dfi.rename( columns = { col : 'transacciones' } , inplace=True)
dfi.columns = dfi.columns.str.replace('mensual_', '', regex=False)

dfi.codigo = pd.to_numeric(dfi['codigo'], errors='coerce')
dfi = dfi[(dfi['codigo'] < 100)]

dfi.drop('codigo', axis=1, inplace=True)


dfi = pd.melt(dfi, id_vars="transacciones", var_name="date")
dfi.date = pd.to_datetime(dfi.date)
dfi.transacciones = dfi.transacciones.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
dfi['transacciones'] = dfi['transacciones'].str.replace(r'\(.*?\)', '', regex=True).str.lower().str.strip().str.replace(" ", "_")

dfi.set_index('date', inplace=True)
dfi = pd.pivot(dfi, columns="transacciones", values="value")

dfi.rename( columns=lambda x: f"gc_{x}" , inplace=True) 
print(f"Updated: {dfi.index.max()}")
dfi.to_csv(f'{PATH_RAW}/mef_gc.csv')

In [ ]:
try :
    os.remove( f"{PATH_RAW}/{target}" )
    print(f"{target} has been deleted.")
except :
    print(f"{target} not found.")

**Deuda Publica**

In [ ]:
df0 = pd.read_csv(f"{PATH_RAW}/mef_deuda.csv", parse_dates=True, index_col='date')

last_date = df0.index.max()

In [ ]:
from selenium import webdriver
import shutil
import time
from selenium.webdriver.chrome.service import Service
download_folder = "C:/Users/guerr/Downloads/"

# URL de la página que contiene los archivos
url = "https://www.finanzas.gob.ec/https-wwwdeuda-publica-nueva-metodologia/"  # Reemplaza con la URL real

# Realizar la solicitud y parsear el HTML
r = requests.get(url=url, verify=False, headers=hdr,timeout=20).content
soup = BeautifulSoup(r, 'html.parser')

# Encontrar todos los enlaces
links = soup.find_all('a')
filtered_links = [ link for link in links if link.text.startswith("Base de Datos") ]
months = {'enero': '01', 'febrero': '02', 'marzo':'03', 'abril':'04', 'mayo': '05', 'junio': '06','julio': '07','agosto': '08','septiembre': '09','octubre': '10','noviembre': '11','diciembre': '12', 
                  'ene': '01', 'feb': '02', 'mar':'03', 'abr':'04', 'may': '05', 'jun': '06','jul': '07','ago': '08','sep': '09','oct': '10','nov': '11','dic': '12','dec': '12' }

# Remover links que ya estan en los datos
links = list()
for x in filtered_links :
    m , y = x.contents[-1].lower().split()[-2], x.contents[-1].lower().split(" ")[-1]
    try :
        m = months[m]
        d = pd.to_datetime(f"01-{m}-{y}", format = "%d-%m-%Y")
        if d > last_date :
            links.append(x['href'])
    except :
        continue

In [ ]:
for link in links :
    driver = webdriver.Chrome()
    driver.get(link)
    time.sleep(10)
    driver.quit()

target = [ f for f in os.listdir(download_folder) if
          ( "base-" in f.lower() or "cuaderno-" in f.lower() or "base" in f.lower() ) 
          and ( 'xls' in f ) ]
if len(target) > 0:
    for f in target:
        shutil.move(f"{download_folder}/{f}", f"{PATH_RAW}/{f}")

Identificar que archivos se necesitan actualizar. Borrar el resto

Actualizacion:

In [ ]:
target = [ f for f in os.listdir(f"{PATH_RAW}") if
          ( "base-" in f.lower() or "cuaderno-" in f.lower() or "base" in f.lower() ) 
          and ( 'xls' in f ) ]


for t in target :
    with pd.ExcelFile(f"{PATH_RAW}/{t}") as xls:
        df = pd.ExcelFile(xls)
    
    mysheet = [ x for x in df.sheet_names if "spnf" in x.lower() and "cons" in x.lower()]
    if len(mysheet)>0 :
        mysheet=mysheet[0]
        
        dfi = pd.read_excel( 
            df ,
            sheet_name=mysheet, 
            skiprows=4
        )
        
        dfi.columns = dfi.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
        dfi['deuda externa'] = dfi['deuda externa'].astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
        dfi = dfi[dfi['deuda externa'].astype(str).str.startswith('total deuda publica')]
        dfi = dfi[[ col for col in dfi.columns if "deuda externa" in col or "saldo al" in col ]]
        
        dfi = pd.melt(dfi, id_vars="deuda externa", var_name="date")
        dfi['date'] = dfi['date'].astype(str).str.replace("-", "/").str.replace(".", "/")
        dfi['date'] = dfi['date'].str.replace(r'[a-z\s+]', r'', regex=True)
        dfi.date = pd.to_datetime(dfi.date)
        dfi.set_index('date', inplace=True)
        dfi[dfi.index==dfi.index.max()]
        
        dfi = pd.pivot(dfi, columns="deuda externa", values="value")
        dfi.rename( columns=lambda x: f"spnf consolidada {x}" , inplace=True) 
    else :
        notfound.append(t)
        
    mysheet = [
        x for x in df.sheet_names if
        "spnf" in x.lower() 
        and "acree" in x.lower()
        and "-F" not in x
    ]
    if len(mysheet)>0 :
        mysheet=mysheet[0]
        
        dfk = pd.read_excel( 
            df ,
            sheet_name=mysheet, 
            skiprows=4
        )
        
        dfk.columns = dfk.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
        dfk['deuda externa'] = dfk['deuda externa'].astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()
        dfk = dfk[dfk['deuda externa'].astype(str).str.startswith('total deuda publica')]
        dfk = dfk[[ col for col in dfk.columns if "deuda externa" in col or "saldo al" in col ]]
        
        dfk = pd.melt(dfk, id_vars="deuda externa", var_name="date")
        dfk['date'] = dfk['date'].astype(str).str.replace("-", "/").str.replace(".", "/")
        dfk['date'] = dfk['date'].str.replace(r'[a-z\s+]', r'', regex=True)
        dfk.date = pd.to_datetime(dfk.date)
        dfk.set_index('date', inplace=True)
        dfk[dfk.index==dfk.index.max()]
        
        dfk = pd.pivot(dfk, columns="deuda externa", values="value")
        dfk.rename( columns=lambda x: f"spnf agg {x}" , inplace=True) 
    
    df = pd.concat( [dfi, dfk ] , axis=1 )
    df0 = pd.concat( [ df0, df ], axis=0).sort_index()
    os.remove(f"{PATH_RAW}/{t}")

df0 = df0.resample('MS').last()
df0.to_csv(f"{PATH_RAW}/mef_deuda.csv")

**COSEDE**

In [ ]:
url = "https://www.cosede.gob.ec/publicaciones-estadisticas-mensuales-pem/"

r = requests.get(url=url, verify=False, headers=hdr).content
soup = BeautifulSoup(r, 'html.parser')
links = soup.find_all('a')

links = [ link.get('href') for link in links if "xlsx" in link.get('href') if "liquidez" in link.get('href').lower() and ( "historico" in link.get('href').lower() or "histórico" in link.get('href').lower() ) ]
links = max(links, key=lambda s: int(re.search(r'PEM-(\d+)-', s).group(1)))

df = pd.ExcelFile(links)
df = pd.read_excel(df, sheet_name='FLSFP', skiprows=9, skipfooter=3)
df.columns = df.columns.astype("str").str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.lower()

df.rename(columns = {'unnamed: 2' : 'year'}, inplace=True)
df = df[[ col for col in df.columns if "unnamed" not in col ]]

df = pd.melt(df, id_vars='year', var_name='month', value_name='cosede_liquidez')
df['month'] = df['month'].astype(str).str.lower().replace(months, regex=True)
df['date'] = pd.to_datetime(df[['year', 'month']].assign(DAY=1), format='%b')
df = df[['cosede_liquidez', 'date']]
df.set_index('date', inplace=True)
df.dropna(inplace=True)
df.sort_index(inplace=True)

print(f"Updated: {df.index.max()}")
df.to_csv(f'{PATH_RAW}/cosede_lfsfp.csv')

## Merge Files

In [5]:
import glob
files = glob.glob(f'{PATH_FORMAT}/*.csv')
for f in files:
    os.remove(f)

dict = pd.read_excel(f'{PATH_RAW}/dict.xlsx')
for col in dict.columns : dict[col] = dict[col].astype(str).str.strip()

defined = dict[['ticket', 'definition', 'source']].set_index('ticket')
defined = pd.DataFrame({'definition' : defined.apply(lambda row: f"Definition: {row['definition']}. Source: {row['source']}", axis=1)})
defined = defined['definition'].to_dict()

dict = pd.read_excel(f'{PATH_RAW}/dict.xlsx')[['ticket', 'var name', 'file']]
file_list = [ csv for csv in os.listdir(f"{PATH_RAW}") if "csv" in csv and csv in dict.file.unique() ]
dict = dict.set_index('var name')['ticket'].to_dict()

for file in file_list :
    dfi = pd.read_csv(f"{PATH_RAW}/{file}" , parse_dates=True, index_col='date')
    dfi.rename(columns=dict , inplace=True)
    dfi = dfi[[col for col in dfi.columns if col in dict.values()]]
    
    for col in dfi.columns:
        dfi[[col]].to_csv(f"{PATH_FORMAT}/{col}.csv", index=True)
        with open(f"{PATH_FORMAT}/{col}.csv", 'a') as output :
            output.write(f"\n\n {defined[col]}\n")

List any file that was not added:

In [ ]:
[ x for x in list(dict.values()) if x+".csv" not in os.listdir(PATH_FORMAT) ]